# Inferring Likelihood Graphs using Next Event Predictors

In this notebook we infer a likelihood graph using the trained next event predictor.

* *Event Log*: The event log for which the likelihood graph should be inferred.
* *Model*: The next event predictor to use. The event log the next event predictor was trained on must match the chosen event log.
* *Heuristic*: How to determine the likelihood thresholds. Manual selection (must set threshold in appropriate field) or one of the lowest plateau heuristics (cf. BINet papers).
* *Threshold*: The likelihood thresholds to use. Separated by commas. Might fail.
* *Activity Count Threshold*: If an activity is repeated this often, the case is discarded.
* *Group Attribute Nodes*: Whether to also group attribute nodes in the likelihood graph.
* *Use Event Attributes*: Whether to also include event attributes in the graph. Must match the used next event predictor.

In [2]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from ipywidgets import widgets, interact, interact_manual, Layout, Button, Box
from IPython.display import display

from april.fs import DATE_FORMAT, EVENTLOG_DIR, MODEL_DIR
from neplg.api import routines

import arrow
import os


event_log_widget = widgets.Dropdown(description='Event Log')

model_name_widget = widgets.Dropdown(description='Model')
threshold_heuristic_widget = widgets.Dropdown(description='Heuristic', options=['manual', 'left', 'mean', 'right'])
next_event_threshold_widget = widgets.Text(descriptions='Likelihood Thresholds')
next_event_threshold_widget_hbox = widgets.HBox([widgets.Label(value="Likelihood Thresholds (separated by commas)"), next_event_threshold_widget])
activity_count_threshold_widget = widgets.Text(descriptions='Activity Count Threshold', value='5')
activity_count_threshold_widget_hbox = widgets.HBox([widgets.Label(value="Activity Count Threshold"), next_event_threshold_widget])
group_attributes_widget = widgets.Checkbox(description='Group Attribute Nodes', value=False)
event_attributes_widget = widgets.Checkbox(description='Use Event Attributes', value=True)

reload_button = widgets.Button(description="Reload")
discover_button = widgets.Button(description="Discover")

first_row = widgets.HBox([event_log_widget, model_name_widget, reload_button])
second_row = widgets.HBox([threshold_heuristic_widget, next_event_threshold_widget_hbox])
third_row = widgets.HBox([group_attributes_widget, event_attributes_widget])
fourth_row = widgets.HBox([discover_button])

parameter_gui = widgets.Output()
output = widgets.Output()

with parameter_gui:
    display(widgets.VBox([first_row, second_row, third_row, widgets.Label(value="Activity Count Threshold"), activity_count_threshold_widget, fourth_row]))

def get_all_event_logs():
    files = os.listdir(EVENTLOG_DIR)
    return [file[0:len(file)-8] for file in files if file[-8:] == '.json.gz']

def get_all_models():
    files = os.listdir(MODEL_DIR)
    return [file[0:len(file)-3] for file in files if file[-3:] == '.h5']
    

def on_reload(button):
    event_log_widget.options = get_all_event_logs()
    model_name_widget.options = get_all_models()


def discover_process_model(button):
    event_log, model, next_event_threshold = event_log_widget.value, model_name_widget.value, next_event_threshold_widget.value 
    threshold_heuristic = threshold_heuristic_widget.value
    group_attribute_nodes, use_event_attributes = group_attributes_widget.value, event_attributes_widget.value
    activity_count_threshold = activity_count_threshold_widget.value
    
    start_time = arrow.now()
    
    output_name = f'results_{event_log}-{model}-{start_time.format(DATE_FORMAT)}-{group_attribute_nodes}-{next_event_threshold}'
    output_locations = [output_name]
    
    output.clear_output()
    with output:
        print(f'The discovered process model will be stored as {output_name}')
        routines.inference(output_locations=output_locations, event_log=event_log, model=model, threshold_heuristic=threshold_heuristic, next_event_threshold=next_event_threshold, group_attribute_nodes=group_attribute_nodes, activity_count_threshold=int(activity_count_threshold))


discover_button.on_click(discover_process_model)
reload_button.on_click(on_reload)
on_reload(None)

display(parameter_gui)
display(output)

Output()

Output()